In [52]:
import sys

sys.path.append("/gstore/project/paneldesign/pkg")

sys.path.append("/gstore/home/taol9/gitsss/yo/tao")

from Bio import SeqIO
from Bio.Seq import Seq
from Bio import Entrez
import csv
from collections import defaultdict,OrderedDict
from collections import Counter
import matplotlib.pyplot as plt

import pandas as pd
import os
import pickle
import random
from myfunc import rc, RandomDNA


In [51]:
# !pip3 install --target /gstore/project/paneldesign/pkg barcode

  Using cached https://files.pythonhosted.org/packages/3f/d2/5560221cbbd54c5ea766ec34d0ce79f92436e1564b51c38001eff7e7bd4f/barcode-1.0.2.tar.gz
  Using cached https://files.pythonhosted.org/packages/c9/bd/fef1536c7ea8d22305c6d54fecf4e7abfc5b4b55782ff9193cdcea4ff8b9/python_Levenshtein-0.20.9-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/22/ea/b2c2c26477111be2938dca870044583f6e785d952c0c424c7ccf0dbe0144/Levenshtein-0.20.9.tar.gz
    Complete output from command python setup.py egg_info:
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "/local/64202154/pip-build-5o3jsi6v/Levenshtein/setup.py", line 1, in <module>
        from skbuild import setup
    ModuleNotFoundError: No module named 'skbuild'
    
    ----------------------------------------
Command "python setup.py egg_info" failed with error code 1 in /local/64202154/pip-build-5o3jsi6v/Levenshtein/


In [ ]:
def bgen(length,maxstretch=3,distance=3,):
    candi=[]
    for i in range(30000):
        x=str(RandomDNA(length))
        if 45<GC(x)<55:
            candi.append(x)
    candi=barcode.filter_stretches(candi, maxstretch)
    candi=barcode.filter_distance(candi, distance)
    return candi

# TSO(25nt)-flankingsequence(25nt)-Index1-UMI-repeat1-spacer(4nt)-repeat2-spacer-repeat3-spacer-repeat4-...repeatX-polyA;

## define adaptor


In [5]:
BC001 = "ACTTTAGG"
BC002 = "AACGGGAA"
BC003 = "AGTAGGCT"
BC004 = "ATGTTGAC"
BC005 = "ACAGACCT"
BC006 = "ATCCCAAC"
BC007 = "AAGTAGAG"
BC008 = "AGCTGTGA"
BC009 = "ACAGTCTG"
BC010 = "AGTGAGTG"
BC011 = "AGAGGCAA"
BC012 = "ACTACTCA"
BC013 = "ATACGTCA"
BC014 = "ATCATGTG"
BC015 = "AACGCCGA"
BC016 = "ATTCGGTT"

Mly1_R_w='GCTTCCTGATGAGTCCGATG'
Mly1_F_w='GCAACGACTCCACACTCATA'

# TSO(25nt)-flankingsequence(25nt)-Index1-UMI-repeat1-spacer(4nt)-repeat2-spacer-repeat3-spacer-repeat4-...repeatX-polyA;

In [61]:
# random combination
# spacer=RandomDNA(4)
# units=["C","TG","AG"]
# str_len=[9,18]
# UMI="N"*6
# index=BC003
# TSO='AAGCAGTGGTATCAACGCAGAGTACAT'
# dialout_adaptor_F=Mly1_F_w='GCAACGACTCCACACTCATA'
# dialout_adaptor_R=Mly1_R_w='GCTTCCTGATGAGTCCGATG'
# l=TSO+dialout_adaptor_F+index+UMI
# n=0
# while len(l)<1500:
#     l=l+random.choice(str_len)*random.choice(units)+spacer
#     n=n+1
# #     print (l)
# print(n)
# l=l+dialout_adaptor_R


    

    
    

In [93]:
#fix length with units combination
def strtracer(index,maxlen=2000,str_len=15,UMIlen=10):
    spacer=RandomDNA(str_len)
    units=["AC","C","AG"]
    UMI="N"*UMIlen
    TSO='AAGCAGTGGTATCAACGCAGAGTACAT'
    dialout_adaptor_F=Mly1_F_w='GCAACGACTCCACACTCATA'
    dialout_adaptor_R=Mly1_R_w='GCTTCCTGATGAGTCCGATG'
    bridge=RandomDNA(25)
    l=partA=TSO+dialout_adaptor_F+index+UMI+bridge
    n=0
    while len(l)<maxlen-len(partA):
        l=l+str_len*random.choice(units)+spacer+str_len*random.choice(units)+spacer+str_len*random.choice(units)+spacer
        n=n+3
#     print (l)
    print(n)
    l=str(l+dialout_adaptor_R)
    return {"index":index,"partA":str(partA), "partB":l[len(partA)-25:],"Full":str(l), "strs_counts":n,"len":len(l)}

In [72]:
indexes='''TGGCACTCAT
ATATGCGCCG
TTACCTACGG
CTTATCCGGA
GCTACTGACG
AGCCAATTGC
AAGTCGTAAC
CTCGGGCAAT
TCCATAGGTG
GAAGTTCATC'''.split()
indexes

['TGGCACTCAT',
 'ATATGCGCCG',
 'TTACCTACGG',
 'CTTATCCGGA',
 'GCTACTGACG',
 'AGCCAATTGC',
 'AAGTCGTAAC',
 'CTCGGGCAAT',
 'TCCATAGGTG',
 'GAAGTTCATC']

In [94]:
output=dict()
for idx in indexes[:4]:
    output["2K_"+str(idx)]=strtracer(index,maxlen=2000,str_len=15,UMIlen=10)
for idx in indexes[-4:]:
    output["5K_"+str(idx)]=strtracer(index,maxlen=5000,str_len=15,UMIlen=10)

45
48
48
48
123
123
120
120


In [95]:

df=pd.DataFrame.from_dict(output,orient='index')


In [96]:
df

,index,partA,partB,Full,strs_counts,len
2K_ATATGCGCCG,AGTAGGCT,AAGCAGTGGTATCAACGCAGAGTACATGCAACGACTCCACACTCAT...,AGATTGCGTCCAGTAGGATAAATGAAGAGAGAGAGAGAGAGAGAGA...,AAGCAGTGGTATCAACGCAGAGTACATGCAACGACTCCACACTCAT...,48,2030
2K_CTTATCCGGA,AGTAGGCT,AAGCAGTGGTATCAACGCAGAGTACATGCAACGACTCCACACTCAT...,AAGTGTATTCAAACGCGCTGTAATTACACACACACACACACACACA...,AAGCAGTGGTATCAACGCAGAGTACATGCAACGACTCCACACTCAT...,48,2015
2K_TGGCACTCAT,AGTAGGCT,AAGCAGTGGTATCAACGCAGAGTACATGCAACGACTCCACACTCAT...,GTTATGTGCGATTCCCTGACTTCCCAGAGAGAGAGAGAGAGAGAGA...,AAGCAGTGGTATCAACGCAGAGTACATGCAACGACTCCACACTCAT...,45,1955
2K_TTACCTACGG,AGTAGGCT,AAGCAGTGGTATCAACGCAGAGTACATGCAACGACTCCACACTCAT...,TGAATTGGTTGCCGCGCCACCGCTCAGAGAGAGAGAGAGAGAGAGA...,AAGCAGTGGTATCAACGCAGAGTACATGCAACGACTCCACACTCAT...,48,2045
5K_AAGTCGTAAC,AGTAGGCT,AAGCAGTGGTATCAACGCAGAGTACATGCAACGACTCCACACTCAT...,GGTCTTAAGTTGTGAAACAGTAGGGCCCCCCCCCCCCCCCGACACT...,AAGCAGTGGTATCAACGCAGAGTACATGCAACGACTCCACACTCAT...,123,5000
5K_CTCGGGCAAT,AGTAGGCT,AAGCAGTGGTATCAACGCAGAGTACATGCAACGACTCCACACTCAT...,GTATGATGTGTGCGTGGTTCTCAAGCCCCCCCCCCCCCCCGTAGAG...,AAGCAGTGGTATCAACGCAGAGTACATGCAACGACTCCACACTCAT...,123,4985
5K_GAAGTTCATC,AGTAGGCT,AAGCAGTGGTATCAACGCAGAGTACATGCAACGACTCCACACTCAT...,ACTACTACAGGCCATTTCCTAGGCGAGAGAGAGAGAGAGAGAGAGA...,AAGCAGTGGTATCAACGCAGAGTACATGCAACGACTCCACACTCAT...,120,4955
5K_TCCATAGGTG,AGTAGGCT,AAGCAGTGGTATCAACGCAGAGTACATGCAACGACTCCACACTCAT...,GCCTACAAAAGCCCACTAAAGATGGAGAGAGAGAGAGAGAGAGAGA...,AAGCAGTGGTATCAACGCAGAGTACATGCAACGACTCCACACTCAT...,120,5030


In [291]:
df.to_csv('str_tracer_230310.csv')

In [ ]:
pwd